In [1]:
import pandas as pd
import numpy as np

In [2]:
#load needed rows and columns into dataframe
df = pd.read_csv('concepts.csv', header=None, nrows=10000)
df.dropna(axis='columns', inplace=True, thresh=9999)

#preprocessing, delete extra row
#from itertools import compress
#tf2 = df[df.columns[0]].isna()
#list(compress(range(len(tf2)), tf2))
#print(tf2[3484])
#print(df.iloc[3484])
df.dropna(axis='rows', inplace=True)
print(df[0])

0                  machine learning
1                 genetic algorithm
2                    classification
3                     deep learning
4                       data mining
                   ...             
9995    empirical risk minimization
9996                  global motion
9997                 retinal images
9998       unequal error protection
9999             dark channel prior
Name: 0, Length: 9999, dtype: object


In [3]:
#parse json, code is from https://www.kaggle.com/srishti280992/prep-data-for-coauthorship-analysis
import json

articles = []
category = 'nucl-ex'
with open("papers.json", "r") as f:
    for l in f:
        d = json.loads(l)
        if category in d['categories'].split(' '):
            articles.append(d)

articles_df = pd.DataFrame().from_records(articles)
articles_df.head()

clean_abstract = []
i = 0
for i,a in articles_df.iterrows():
    # Clean abstract
    i += 1
    if i == 10000:
        break
    try:
        clean_abstract.append(LatexNodes2Text().latex_to_text(a['abstract']).replace('\n', ' ').strip()) 
    except:
        clean_abstract.append(a['abstract'].replace('\n', ' ').strip())
#clean_abstract[0]
#articles_df['abstract'][0]

In [4]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/ahsangilani/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [5]:
#load abstracts into whoosh database
from whoosh.fields import Schema, TEXT, ID
from whoosh import index

schema = Schema(content=TEXT(stored=True))
ix = index.create_in(".", schema)
writer = ix.writer()
for i in range(len(clean_abstract)):
    sents = nltk.tokenize.sent_tokenize(clean_abstract[i])
    for j in range(len(sents)):
        writer.add_document(content = sents[j])   
writer.commit()

In [6]:
from whoosh.qparser import QueryParser
 
def search(term):
    ret = np.array([])
    
    with ix.searcher() as searcher:
        query = QueryParser("content", ix.schema).parse(term)
        results = searcher.search(query, terms=True, limit = 100)
        for r in results:
            ret = np.append(ret, r['content'])
        
    return ret

In [7]:
term = "experiment"
matches = search(term)
matches

array(['This experiment is a continuation of the currently running SeaQuest experiment.',
       'The NA60 experiment is a fixed-target experiment at the CERN SPS.',
       'ALICE (A Large Ion Collider Experiment) is the LHC experiment dedicated to the study of the QGP.',
       'ALICE (A Large Ion Collider Experiment) is the dedicated heavy-ion experiment at the LHC.',
       'The design of the experiment is presented.',
       'The detail of the experiment is described.',
       'Data of an early experiment and the recent Crystal Ball experiment at BNL are included in the analysis with the c.m.',
       "The results of the experiment's commissioning run are reported here, constituting approximately 4% of the data collected in the experiment.",
       'The Main Injector Particle Production (MIPP) experiment is a fixed target hadron production experiment at Fermilab.',
       'ALICE (A Large Ion Collider Experiment) is the dedicated heavy ion experiment at the Large Hadron Collider at 

In [8]:
concepts = dict((k,0) for k in df[0])
outputs = dict((k,"") for k in df[0])

def output(concepts, outputs):
    o = list(reversed(sorted(concepts.items(), key=lambda item: item[1])))
    #print(o)
    print(outputs[o[0][0]])
    print(outputs[o[1][0]])
    print(outputs[o[2][0]])

def rank(term, matches, scores, outputs):
    for sent in matches:
        for k in scores:
            words = sent.split()
            if k in words:
                scores[k] += 1
                i1 = words.index(k)
                i2 = [idx for idx, s in enumerate(words) if term in s][0]
                scores[k] += len(words) - abs(i1 - i2)
                if outputs[k] == "":
                    outputs[k] = sent
                    
    output(scores, outputs)
    return

In [9]:
rank(term, matches, concepts, outputs)

This experiment is a continuation of the currently running SeaQuest experiment.
The Main Injector Particle Production (MIPP) experiment is a fixed target hadron production experiment at Fermilab.
The results of the experiment's commissioning run are reported here, constituting approximately 4% of the data collected in the experiment.
